In [1]:
import os
# перейдем в родительскую директорию чтобы использовать пакет raiflib без установки
notebook_path = os.getcwd()
os.chdir(os.path.dirname(notebook_path))

In [2]:
import re
import pandas as pd
import numpy as np

from tqdm import tqdm
from tqdm.auto import tqdm
tqdm.pandas()

In [3]:
df = pd.read_csv('../data/train.csv')
tdf = pd.read_csv('../data/test.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Исправим ошибки, найденные в ходе исследования данных

In [4]:
# 1. Город Санкт-Петербург в данных представлен для двух регионов
df[df.city == 'Санкт-Петербург'].region.value_counts()

Санкт-Петербург          18888
Ленинградская область       18
Name: region, dtype: int64

In [5]:
df['region'] = df['region'].where(df.city != 'Санкт-Петербург', 'Санкт-Петербург')
tdf['region'] = tdf['region'].where(tdf.city != 'Санкт-Петербург', 'Санкт-Петербург')
df[df.city == 'Санкт-Петербург'].region.value_counts()

Санкт-Петербург    18906
Name: region, dtype: int64

# Очистка данных floor
## Заметим большое число пропусков в floor. Тем не менее, данный параметр зачастую очень важен при оценке недвижимости. Попытаемся навести порядок в этой переменной. 

In [8]:
def get_int_floor(string) -> int:
    if re.search('(?<![0-9\-])1(?![0-9])', string):
        return 1
    elif re.search('(подвал|цоколь|-1)', string):
        return -1
    else:
        try:
            return re.search('(?<![0-9])-*\d+(?![0-9])', string).group()
        except:
                return 0 #  вставить паттерн неизвестного значения
    
def categorize_floor(floor) -> int:
    if isinstance(floor, str):
        floor = get_int_floor(floor)

    try:
        floor = int(floor)
    except:
        return 0

    if floor < 0:
        return -1
    elif floor == 1 or floor == 2:
        return floor
    elif floor in range(3,6):
        return 3
    elif floor in range(5,11):
        return 4
    elif floor in range(10,21):
        return 5
    elif floor in range(20,51):
        return 6
    elif floor > 50:
        return 7
    else:
        return 0
    

def prepare_floor(floors: pd.Series) -> pd.Series:
    return floors.progress_apply(categorize_floor)

In [9]:
df['floor'] = prepare_floor(df.floor).astype('int32')
df['reform_mean_floor_count_1000'] = prepare_floor(df.reform_mean_floor_count_1000).astype('int32')
tdf['floor'] = prepare_floor(tdf.floor).astype('int32')
tdf['reform_mean_floor_count_1000'] = prepare_floor(tdf.reform_mean_floor_count_1000).astype('int32')

100%|██████████| 2974/2974 [00:00<00:00, 250403.70it/s]


# Очистка от нерепрезентативных объявлений

In [10]:
# Обратив внимание на города, в которых было найдено только одно-три объявления, 
# удалим таковые из предположения, что нельзя определить, адекватное было объявление или нет.

city_offer_counts = df[df.price_type == 0] \
        .groupby(['region', 'city'], as_index=False) \
        .agg({'id': 'count'})

city_offer_indeces_series = df.reset_index()[df.price_type == 0] \
    .groupby(['region', 'city'])['index'].apply(list)
city_offer_indeces = pd.DataFrame(city_offer_indeces_series.index.tolist(), columns=['region', 'city'])
city_offer_indeces['indeces'] = city_offer_indeces_series.tolist()
low_city_offers = pd.merge(city_offer_counts, city_offer_indeces, on=['region', 'city']).query('id < 5')
bad_idx_list = [idx for city_indeces in low_city_offers.indeces.tolist() for idx in city_indeces]
bad_df = df.index.isin(bad_idx_list)
df = df[~bad_df]

# Очистка данных reform

Среди данных reform замечены такие недочеты, как:
- пропуски
```
df[['reform_mean_year_building_500', 'reform_mean_year_building_1000']].info()
```
- выбросы
```
df[['city','id','reform_mean_year_building_500', 'reform_mean_year_building_1000']][df.reform_mean_year_building_500.isna()] \
    .sort_values(by='reform_mean_year_building_1000', na_position='first').query('reform_mean_year_building_1000 < 1600')
```



In [11]:
# 1. Значения меньше 1600 сделать NaN
df['reform_mean_year_building_1000'] = df['reform_mean_year_building_1000'].where(df['reform_mean_year_building_1000'] >= 1600, np.NaN)
tdf['reform_mean_year_building_1000'] = tdf['reform_mean_year_building_1000'].where(tdf['reform_mean_year_building_1000'] >= 1600, np.NaN)

# 2. Вычисляем медианные значения для каждого города
tmp = df[['region', 'city', 'reform_mean_year_building_1000']] \
        .dropna() \
        .groupby(['region', 'city'], as_index=False)['reform_mean_year_building_1000'] \
        .median() \
        .rename(columns={'reform_mean_year_building_1000': 'city_median_reform_mean_year_building_1000'})
df = pd.merge(df, tmp, on=['region', 'city'], how='left')
tdf = pd.merge(tdf, tmp, on=['region', 'city'], how='left')

# 3. Вычисляем медианные значения для каждого региона
tmp = df[['region', 'reform_mean_year_building_1000']] \
        .dropna() \
        .groupby(['region'], as_index=False)['reform_mean_year_building_1000'] \
        .median() \
        .rename(columns={'reform_mean_year_building_1000': 'region_median_reform_mean_year_building_1000'})
df = pd.merge(df, tmp, on=['region'], how='left')
tdf = pd.merge(tdf, tmp, on=['region'], how='left')

# Заполнение пропусков в колонке reform_mean_year_building.
# Создаем колонку reform_mean_year_building с значениями reform_mean_year_building_500, 
# NaN значения в которых заполняем значениями 1км, города, региона последовательно
df['reform_mean_year_building'] = df['reform_mean_year_building_500'].where(df.reform_mean_year_building_500.notna(), df.reform_mean_year_building_1000)
df['reform_mean_year_building'] = df['reform_mean_year_building'].where(df.reform_mean_year_building.notna(), df.city_median_reform_mean_year_building_1000)
df['reform_mean_year_building'] = df['reform_mean_year_building'].where(df.reform_mean_year_building.notna(), df.region_median_reform_mean_year_building_1000)

tdf['reform_mean_year_building'] = tdf['reform_mean_year_building_500'].where(tdf.reform_mean_year_building_500.notna(), tdf.reform_mean_year_building_1000)
tdf['reform_mean_year_building'] = tdf['reform_mean_year_building'].where(tdf.reform_mean_year_building.notna(), tdf.city_median_reform_mean_year_building_1000)
tdf['reform_mean_year_building'] = tdf['reform_mean_year_building'].where(tdf.reform_mean_year_building.notna(), tdf.region_median_reform_mean_year_building_1000)

# Очистка объявлений от выбросов в per_square_meter_price

In [12]:
# сначала составить минимальный датафрейм для анализа, а затем его потихоньку расширять
# ['date', 'id','city', 'region', 'realty_type', 'total_square', 'per_square_meter_price', 'floor', 'lat', 'lng', 'reform_mean_year_building_*']
# - floor
# - reform_mean_year_building

In [13]:
# вручную удалим повторяющиеся признаки с разным метражом, оставив только значение "in 0.01" для osm и "1000" для reform
main_columns = [
    'id',
    'region', 
    'city', 'floor',
    'total_square', 
    'realty_type', 
    'price_type', 
    'osm_amenity_points_in_0.01', 
    'osm_building_points_in_0.01', 
    'osm_catering_points_in_0.01', 
    'osm_city_closest_dist', 
    'osm_city_nearest_name', 
    'osm_city_nearest_population',
    'osm_crossing_closest_dist', 
    'osm_crossing_points_in_0.01', 
    'osm_culture_points_in_0.01',
    'osm_healthcare_points_in_0.01', 
    'osm_historic_points_in_0.01', 
    'osm_hotels_points_in_0.01',
    'osm_leisure_points_in_0.01', 
    'osm_offices_points_in_0.01', 
    'osm_shops_points_in_0.01', 
    'osm_subway_closest_dist',
    'osm_train_stop_closest_dist', 
    'osm_train_stop_points_in_0.01', 
    'osm_transport_stop_closest_dist',
    'osm_transport_stop_points_in_0.01', 
    'reform_count_of_houses_1000', 
    'reform_house_population_1000',
    'reform_mean_floor_count_1000',
    'reform_mean_year_building',
    'lat', 'lng',
    'date'
]

In [14]:
df[main_columns].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274280 entries, 0 to 274279
Data columns (total 34 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 274280 non-null  object 
 1   region                             274280 non-null  object 
 2   city                               274280 non-null  object 
 3   floor                              274280 non-null  int32  
 4   total_square                       274280 non-null  float64
 5   realty_type                        274280 non-null  int64  
 6   price_type                         274280 non-null  int64  
 7   osm_amenity_points_in_0.01         274280 non-null  int64  
 8   osm_building_points_in_0.01        274280 non-null  int64  
 9   osm_catering_points_in_0.01        274280 non-null  int64  
 10  osm_city_closest_dist              274280 non-null  float64
 11  osm_city_nearest_name              2742

In [15]:

tdf[main_columns].to_csv('../data/1.ipynb_Cleaning_data_test.csv')
main_columns.append('per_square_meter_price')
df[main_columns].to_csv('../data/1.ipynb_Cleaning_data.csv')

In [16]:
# TODO: доочистить данные
#           - убрать выбросы по lat, lon